In [1]:
import pandas as pd
data = pd.read_json('cladder.json')
data = data.dropna().reset_index(drop=True)

#create column "formal_form" and "graph" that holds the value corresponding to the key "formal_form" at the dict inside column "meta"

data = data.assign(formal_form = lambda x: (x['meta']))
df = data['formal_form'].apply(lambda x: (x.get("formal_form")))
data['formal_form'] = df

data = data.assign(graph = lambda x: (x['reasoning']))
df = data['graph'].apply(lambda x: (x.get("step1")))
data['graph'] = df

data = data.assign(query_type = lambda x: (x['meta']))
df = data['query_type'].apply(lambda x: (x.get("query_type")))
data['query_type'] = df

data = data.assign(info = lambda x: (x['meta']))
df = data['info'].apply(lambda x: (x.get("given_info")))
data['info'] = df
data


,question_id,desc_id,given_info,question,answer,meta,reasoning,formal_form,graph,query_type,info
0,4,alarm-mediation-nde-model0-spec0-q0,For husbands that don't set the alarm and wive...,If we disregard the mediation effect through w...,yes,"{'story_id': 'alarm', 'graph_id': 'mediation',...",{'step0': 'Let X = husband; V2 = wife; Y = ala...,"E[Y_{X=1, V2=0} - Y_{X=0, V2=0}]","X->V2,X->Y,V2->Y",nde,"{'p(Y | X, V2)': [[0.08430222457648501, 0.5394..."
1,7,alarm-mediation-ate-model1-spec1-q1,"For husbands that don't set the alarm, the pro...",Will alarm set by husband decrease the chance ...,no,"{'story_id': 'alarm', 'graph_id': 'mediation',...",{'step0': 'Let X = husband; V2 = wife; Y = ala...,E[Y | do(X = 1)] - E[Y | do(X = 0)],"X->V2,X->Y,V2->Y",ate,"{'p(Y | X)': [0.26076060984306904, 0.755829649..."
2,8,alarm-mediation-marginal-model1-spec1-q0,The overall probability of alarm set by husban...,Is ringing alarm more likely than silent alarm...,yes,"{'story_id': 'alarm', 'graph_id': 'mediation',...",{'step0': 'Let X = husband; V2 = wife; Y = ala...,P(Y),"X->V2,X->Y,V2->Y",marginal,"{'P(X)': 0.771884206668314, 'P(Y | X)': [0.260..."
3,15,alarm-mediation-ate-model3-spec3-q1,"For husbands that don't set the alarm, the pro...",Will alarm set by husband decrease the chance ...,no,"{'story_id': 'alarm', 'graph_id': 'mediation',...",{'step0': 'Let X = husband; V2 = wife; Y = ala...,E[Y | do(X = 1)] - E[Y | do(X = 0)],"X->V2,X->Y,V2->Y",ate,"{'p(Y | X)': [0.19568691380685402, 0.683664327..."
4,21,alarm-mediation-nie-model4-spec4-q0,For husbands that don't set the alarm and wive...,Does husband positively affect alarm clock thr...,no,"{'story_id': 'alarm', 'graph_id': 'mediation',...",{'step0': 'Let X = husband; V2 = wife; Y = ala...,"E[Y_{X=0, V2=1} - Y_{X=0, V2=0}]","X->V2,X->Y,V2->Y",nie,"{'p(Y | X, V2)': [[0.107877091110262, 0.602626..."
...,...,...,...,...,...,...,...,...,...,...,...
8527,31012,nonsense9-fork-det-counterfactual-model3842-sp...,We know that zuph or jyka causes glimx. We obs...,Would an individual is glimx if not zuph inste...,yes,"{'story_id': 'nonsense9', 'graph_id': 'fork', ...",{'step0': 'Let V2 = jyka; X = zuph; Y = glimx....,Y_{X=0} = 1 | V2=1,"X->Y,V2->Y",det-counterfactual,{'V2': 1}
8528,31014,nonsense9-fork-det-counterfactual-model3842-sp...,We know that zuph or jyka causes glimx. We obs...,Would an individual is glimx if zuph instead o...,yes,"{'story_id': 'nonsense9', 'graph_id': 'fork', ...",{'step0': 'Let V2 = jyka; X = zuph; Y = glimx....,Y_{X=1} = 1 | V2=1,"X->Y,V2->Y",det-counterfactual,{'V2': 1}
8529,31015,nonsense9-fork-det-counterfactual-model3842-sp...,We know that zuph or jyka causes glimx. We obs...,Would an individual is not glimx if zuph inste...,no,"{'story_id': 'nonsense9', 'graph_id': 'fork', ...",{'step0': 'Let V2 = jyka; X = zuph; Y = glimx....,Y_{X=1} = 0 | V2=1,"X->Y,V2->Y",det-counterfactual,{'V2': 1}
8530,31016,nonsense9-fork-det-counterfactual-model3843-sp...,We know that zuph and jyka causes glimx. We ob...,Would an individual is glimx if not zuph inste...,no,"{'story_id': 'nonsense9', 'graph_id': 'fork', ...",{'step0': 'Let V2 = jyka; X = zuph; Y = glimx....,Y_{X=0} = 1 | V2=0,"X->Y,V2->Y",det-counterfactual,{'V2': 0}


In [27]:
data['query_type'].unique()

array(['nde', 'ate', 'marginal', 'nie', 'ett', 'correlation', 'exp_away',
       'collider_bias', 'det-counterfactual'], dtype=object)

In [2]:
#exchange "formal_form" and "graph" column's variables to their respective "true name" (ex: exchange X to husband, V2 to wife, etc)
def exchange(x, form, aux):
    x = x.get('step0')
    if(aux == 1):
        form = form.get('formal_form')
    elif(aux == 0):
        form = form.get('step1')
    else:
        form = str(form)
    
    x = x[4:len(x)-1] #erase "Let " and the last "."
    x = x.split('; ')

    for i in range(len(x)):
        aux = x[i].split(" = ")
        form = form.replace(aux[0],aux[1])
    return form


#exchange query_type abbreviation to the whole word

def expand(abb):
    if abb == 'nde':
        return 'natural direct effect'
    elif abb == 'ate':
        return 'average treatment effect'
    elif abb == 'marginal':
        return 'marginal probability'
    elif abb == 'nie':
        return 'natural indirect effect'
    elif abb == 'det-counterfactual':
        return 'normal counterfactual question'
    elif abb == 'ett':
        return 'average treatment effect on treated'
    elif abb == 'correlation':
        return 'conditional probability'
    elif abb == 'exp_away':
        return 'explaining away effect'
    elif abb == 'collider_bias':
        return 'collider bias'
    elif abb == 'backadj':
        return 'backdoor adjustment set'
    return 0
    

data['formal_form'] = data.apply(lambda x: exchange(x['reasoning'], x['meta'],1), axis=1)
data['graph'] = data.apply(lambda x: exchange(x['reasoning'], x['reasoning'],0), axis=1)
data['info'] = data.apply(lambda x: exchange(x['reasoning'], x['info'],2), axis=1)
data['query_type'] = data.apply(lambda x: expand(x['query_type']), axis=1)

data

,question_id,desc_id,given_info,question,answer,meta,reasoning,formal_form,graph,query_type,info
0,4,alarm-mediation-nde-model0-spec0-q0,For husbands that don't set the alarm and wive...,If we disregard the mediation effect through w...,yes,"{'story_id': 'alarm', 'graph_id': 'mediation',...",{'step0': 'Let X = husband; V2 = wife; Y = ala...,"E[alarm clock_{husband=1, wife=0} - alarm cloc...","husband->wife,husband->alarm clock,wife->alarm...",natural direct effect,"{'p(alarm clock | husband, wife)': [[0.0843022..."
1,7,alarm-mediation-ate-model1-spec1-q1,"For husbands that don't set the alarm, the pro...",Will alarm set by husband decrease the chance ...,no,"{'story_id': 'alarm', 'graph_id': 'mediation',...",{'step0': 'Let X = husband; V2 = wife; Y = ala...,E[alarm clock | do(husband = 1)] - E[alarm clo...,"husband->wife,husband->alarm clock,wife->alarm...",average treatment effect,{'p(alarm clock | husband)': [0.26076060984306...
2,8,alarm-mediation-marginal-model1-spec1-q0,The overall probability of alarm set by husban...,Is ringing alarm more likely than silent alarm...,yes,"{'story_id': 'alarm', 'graph_id': 'mediation',...",{'step0': 'Let X = husband; V2 = wife; Y = ala...,P(alarm clock),"husband->wife,husband->alarm clock,wife->alarm...",marginal probability,"{'P(husband)': 0.771884206668314, 'P(alarm clo..."
3,15,alarm-mediation-ate-model3-spec3-q1,"For husbands that don't set the alarm, the pro...",Will alarm set by husband decrease the chance ...,no,"{'story_id': 'alarm', 'graph_id': 'mediation',...",{'step0': 'Let X = husband; V2 = wife; Y = ala...,E[alarm clock | do(husband = 1)] - E[alarm clo...,"husband->wife,husband->alarm clock,wife->alarm...",average treatment effect,{'p(alarm clock | husband)': [0.19568691380685...
4,21,alarm-mediation-nie-model4-spec4-q0,For husbands that don't set the alarm and wive...,Does husband positively affect alarm clock thr...,no,"{'story_id': 'alarm', 'graph_id': 'mediation',...",{'step0': 'Let X = husband; V2 = wife; Y = ala...,"E[alarm clock_{husband=0, wife=1} - alarm cloc...","husband->wife,husband->alarm clock,wife->alarm...",natural indirect effect,"{'p(alarm clock | husband, wife)': [[0.1078770..."
...,...,...,...,...,...,...,...,...,...,...,...
8527,31012,nonsense9-fork-det-counterfactual-model3842-sp...,We know that zuph or jyka causes glimx. We obs...,Would an individual is glimx if not zuph inste...,yes,"{'story_id': 'nonsense9', 'graph_id': 'fork', ...",{'step0': 'Let V2 = jyka; X = zuph; Y = glimx....,glimx_{zuph=0} = 1 | jyka=1,"zuph->glimx,jyka->glimx",normal counterfactual question,{'jyka': 1}
8528,31014,nonsense9-fork-det-counterfactual-model3842-sp...,We know that zuph or jyka causes glimx. We obs...,Would an individual is glimx if zuph instead o...,yes,"{'story_id': 'nonsense9', 'graph_id': 'fork', ...",{'step0': 'Let V2 = jyka; X = zuph; Y = glimx....,glimx_{zuph=1} = 1 | jyka=1,"zuph->glimx,jyka->glimx",normal counterfactual question,{'jyka': 1}
8529,31015,nonsense9-fork-det-counterfactual-model3842-sp...,We know that zuph or jyka causes glimx. We obs...,Would an individual is not glimx if zuph inste...,no,"{'story_id': 'nonsense9', 'graph_id': 'fork', ...",{'step0': 'Let V2 = jyka; X = zuph; Y = glimx....,glimx_{zuph=1} = 0 | jyka=1,"zuph->glimx,jyka->glimx",normal counterfactual question,{'jyka': 1}
8530,31016,nonsense9-fork-det-counterfactual-model3843-sp...,We know that zuph and jyka causes glimx. We ob...,Would an individual is glimx if not zuph inste...,no,"{'story_id': 'nonsense9', 'graph_id': 'fork', ...",{'step0': 'Let V2 = jyka; X = zuph; Y = glimx....,glimx_{zuph=0} = 1 | jyka=0,"zuph->glimx,jyka->glimx",normal counterfactual question,{'jyka': 0}


In [19]:
pd.set_option('display.max_colwidth', None)

In [22]:
#create columnn with complete prompt to be given to the LLM

prompt_start = "Guidance: Address the question above by following the steps below:\nStep 1) Extract the causal graph: Identify the causal graph that depicts the relationships in the scenario. The diagram should simply consist of edges denoted in 'var1 -> var2' format, separated by commas. Step 2) Determine the query type: Identify the type of query implied by the main question. Choices include 'marginal probability', 'conditional probability', 'explaining away effect', 'backdoor adjustment set', 'average treatment effect', 'collider bias', 'normal counterfactual question', 'average treatment effect on treated', 'natural direct effect' or 'natural indirect effect'. Your answer should only be a term from the list above, enclosed in quotation marks. Step 3) Formalize the query: Translate the query into its formal mathematical expression based on its type, utilizing the 'do(·)' notation or counterfactual notations as needed.\nQuestion: "

data = data.assign(prompt = lambda x: (prompt_start+x['given_info']+x['question']))
df = data[['prompt', 'formal_form', 'graph', 'query_type']]
df

,prompt,formal_form,graph,query_type
0,"Guidance: Address the question above by following the steps below:\nStep 1) Extract the causal graph: Identify the causal graph that depicts the relationships in the scenario. The diagram should simply consist of edges denoted in 'var1 -> var2' format, separated by commas. Step 2) Determine the query type: Identify the type of query implied by the main question. Choices include 'marginal probability', 'conditional probability', 'explaining away effect', 'backdoor adjustment set', 'average treatment effect', 'collider bias', 'normal counterfactual question', 'average treatment effect on treated', 'natural direct effect' or 'natural indirect effect'. Your answer should only be a term from the list above, enclosed in quotation marks. Step 3) Formalize the query: Translate the query into its formal mathematical expression based on its type, utilizing the 'do(·)' notation or counterfactual notations as needed.\nQuestion: For husbands that don't set the alarm and wives that don't set the alarm, the probability of ringing alarm is 8%. For husbands that don't set the alarm and wives that set the alarm, the probability of ringing alarm is 54%. For husbands that set the alarm and wives that don't set the alarm, the probability of ringing alarm is 41%. For husbands that set the alarm and wives that set the alarm, the probability of ringing alarm is 86%. For husbands that don't set the alarm, the probability of alarm set by wife is 74%. For husbands that set the alarm, the probability of alarm set by wife is 24%.If we disregard the mediation effect through wife, would husband positively affect alarm clock?","E[alarm clock_{husband=1, wife=0} - alarm clock_{husband=0, wife=0}]","husband->wife,husband->alarm clock,wife->alarm clock",natural direct effect
1,"Guidance: Address the question above by following the steps below:\nStep 1) Extract the causal graph: Identify the causal graph that depicts the relationships in the scenario. The diagram should simply consist of edges denoted in 'var1 -> var2' format, separated by commas. Step 2) Determine the query type: Identify the type of query implied by the main question. Choices include 'marginal probability', 'conditional probability', 'explaining away effect', 'backdoor adjustment set', 'average treatment effect', 'collider bias', 'normal counterfactual question', 'average treatment effect on treated', 'natural direct effect' or 'natural indirect effect'. Your answer should only be a term from the list above, enclosed in quotation marks. Step 3) Formalize the query: Translate the query into its formal mathematical expression based on its type, utilizing the 'do(·)' notation or counterfactual notations as needed.\nQuestion: For husbands that don't set the alarm, the probability of ringing alarm is 26%. For husbands that set the alarm, the probability of ringing alarm is 76%.Will alarm set by husband decrease the chance of ringing alarm?",E[alarm clock | do(husband = 1)] - E[alarm clock | do(husband = 0)],"husband->wife,husband->alarm clock,wife->alarm clock",average treatment effect
2,"Guidance: Address the question above by following the steps below:\nStep 1) Extract the causal graph: Identify the causal graph that depicts the relationships in the scenario. The diagram should simply consist of edges denoted in 'var1 -> var2' format, separated by commas. Step 2) Determine the query type: Identify the type of query implied by the main question. Choices include 'marginal probability', 'conditional probability', 'explaining away effect', 'backdoor adjustment set', 'average treatment effect', 'collider bias', 'normal counterfactual question', 'average treatment effect on treated', 'natural direct effect' or 'natural indirect effect'. Your answer should only be a term from the list above, enclosed in quotation marks. Step 3) Formalize the query: Translate the query into its formal mathematical expression based on its type, utilizing the 'do(·)' notation or coun

In [23]:
#transform the dataset into an "assistant type"

df = df.assign(text = lambda x: ('### Human: ' + x['prompt'] + '### Assistant: Step 1) ' + x['graph'] + '. Step 2) ' + 
                                            x['query_type'] + '. Step 3) ' + x['formal_form'] + '.'))

final_dataset = df[['text']]
final_dataset

,text
0,"### Human: Guidance: Address the question above by following the steps below:\nStep 1) Extract the causal graph: Identify the causal graph that depicts the relationships in the scenario. The diagram should simply consist of edges denoted in 'var1 -> var2' format, separated by commas. Step 2) Determine the query type: Identify the type of query implied by the main question. Choices include 'marginal probability', 'conditional probability', 'explaining away effect', 'backdoor adjustment set', 'average treatment effect', 'collider bias', 'normal counterfactual question', 'average treatment effect on treated', 'natural direct effect' or 'natural indirect effect'. Your answer should only be a term from the list above, enclosed in quotation marks. Step 3) Formalize the query: Translate the query into its formal mathematical expression based on its type, utilizing the 'do(·)' notation or counterfactual notations as needed.\nQuestion: For husbands that don't set the alarm and wives that don't set the alarm, the probability of ringing alarm is 8%. For husbands that don't set the alarm and wives that set the alarm, the probability of ringing alarm is 54%. For husbands that set the alarm and wives that don't set the alarm, the probability of ringing alarm is 41%. For husbands that set the alarm and wives that set the alarm, the probability of ringing alarm is 86%. For husbands that don't set the alarm, the probability of alarm set by wife is 74%. For husbands that set the alarm, the probability of alarm set by wife is 24%.If we disregard the mediation effect through wife, would husband positively affect alarm clock?### Assistant: Step 1) husband->wife,husband->alarm clock,wife->alarm clock. Step 2) natural direct effect. Step 3) E[alarm clock_{husband=1, wife=0} - alarm clock_{husband=0, wife=0}]."
1,"### Human: Guidance: Address the question above by following the steps below:\nStep 1) Extract the causal graph: Identify the causal graph that depicts the relationships in the scenario. The diagram should simply consist of edges denoted in 'var1 -> var2' format, separated by commas. Step 2) Determine the query type: Identify the type of query implied by the main question. Choices include 'marginal probability', 'conditional probability', 'explaining away effect', 'backdoor adjustment set', 'average treatment effect', 'collider bias', 'normal counterfactual question', 'average treatment effect on treated', 'natural direct effect' or 'natural indirect effect'. Your answer should only be a term from the list above, enclosed in quotation marks. Step 3) Formalize the query: Translate the query into its formal mathematical expression based on its type, utilizing the 'do(·)' notation or counterfactual notations as needed.\nQuestion: For husbands that don't set the alarm, the probability of ringing alarm is 26%. For husbands that set the alarm, the probability of ringing alarm is 76%.Will alarm set by husband decrease the chance of ringing alarm?### Assistant: Step 1) husband->wife,husband->alarm clock,wife->alarm clock. Step 2) average treatment effect. Step 3) E[alarm clock | do(husband = 1)] - E[alarm clock | do(husband = 0)]."
2,"### Human: Guidance: Address the question above by following the steps below:\nStep 1) Extract the causal graph: Identify the causal graph that depicts the relationships in the scenario. The diagram should simply consist of edges denoted in 'var1 -> var2' format, separated by commas. Step 2) Determine the query type: Identify the type of query implied by the main question. Choices include 'marginal probability', 'conditional probability', 'explaining away effect', 'backdoor adjustment set', 'average treatment effect', 'collider bias', 'normal counterfactual question', 'average treatment effect on treated', 'natural direct effect' or 'natural indirect effect'. Your answer should only be a term from the list above, enclosed in quotation marks. Step 3) Formalize the query: Translate the query into its formal mat

In [5]:
from huggingface_hub import notebook_login
notebook_login()

In [33]:
#import transformers
#import torch

#model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

#pipeline = transformers.pipeline(
#    "text-generation",
#    model=model_id,
#    model_kwargs={"torch_dtype": torch.bfloat16},
#    device_map="auto",
#)

#messages = [
#    {"role": "system", "content": "You are a chatbot"},
#    {"role": "user", "content": "Who are you?"},
#]

#prompt = pipeline.tokenizer.apply_chat_template(
#        messages, 
#        tokenize=False, 
#        add_generation_prompt=True
#)

#terminators = [
#    pipeline.tokenizer.eos_token_id,
#    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
#]

#outputs = pipeline(
#    prompt,
#    max_new_tokens=256,
#    eos_token_id=terminators,
#    do_sample=True,
#    temperature=0.6,
#    top_p=0.9,
#)
#print(outputs[0]["generated_text"][len(prompt):])


In [34]:
#pipeline("Hey how are you doing today?")

In [35]:
#pipeline.device

In [36]:
#round(0.23519324071521203,2)

In [37]:
#the following code is from this website:
#https://deci.ai/blog/fine-tune-llama-2-with-lora-for-question-answering/

In [6]:
import os, torch, logging
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, HfArgumentParser, TrainingArguments, pipeline
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [24]:
# Dataset
#Transform dataset from DataFrame to Dataset
import datasets
from datasets import Dataset, DatasetDict

final_dataset = final_dataset.sample(frac=1).reset_index(drop=True)
final_dataset = Dataset.from_pandas(final_dataset)
split_form = final_dataset.train_test_split(train_size=0.9, seed=20)
training_data = split_form['train']

# Model and tokenizer names
base_model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
refined_model = "/media/data/bazaluk/cladder_llama3_model"

# Tokenizer
llama_tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
llama_tokenizer.pad_token = llama_tokenizer.eos_token
llama_tokenizer.padding_side = "right"  # Fix for fp16

# Quantization Config
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False
)

# Model
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=quant_config,
    device_map={"": 0}
)
base_model.config.use_cache = False
base_model.config.pretraining_tp = 1

/home/bazaluk/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [25]:
# LoRA Config
peft_parameters = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=8,
    bias="none",
    task_type="CAUSAL_LM"
)

# Training Params
train_params = TrainingArguments(
    output_dir="/media/data/bazaluk/results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant"
)

# Trainer
fine_tuning = SFTTrainer(
    model=base_model,
    train_dataset=training_data,
    peft_config=peft_parameters,
    dataset_text_field="text",
    tokenizer=llama_tokenizer,
    args=train_params
)

# Training
fine_tuning.train()

# Save Model
fine_tuning.model.save_pretrained(refined_model)

/home/bazaluk/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:246: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/7678 [00:00<?, ? examples/s]

Step,Training Loss
25,1.262100
50,0.380200
75,0.344700
100,0.245500
125,0.271300


/home/bazaluk/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/bazaluk/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/bazaluk/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/bazaluk/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will b

OutOfMemoryError: CUDA out of memory. Tried to allocate 1002.00 MiB. GPU 

In [8]:
split_form['test'][0]

{'text': "### Human: Question: We know that zuph or jyka causes glimx. We observed an individual is jyka.Would an individual is not glimx if zuph instead of not zuph?\nGuidance: Address the question above by following the steps below:\nStep 1) Extract the causal graph: Identify the causal graph that depicts the relationships in the scenario. The diagram should simply consist of edges denoted in 'var1 -> var2' format, separated by commas. Step 2) Determine the query type: Identify the type of query implied by the main question. Choices include 'marginal probability', 'conditional probability', 'explaining away effect', 'backdoor adjustment set', 'average treatment effect', 'collider bias', 'normal counterfactual question', 'average treatment effect on treated', 'natural direct effect' or 'natural indirect effect'. Your answer should only be a term from the list above, enclosed in quotation marks. Step 3) Formalize the query: Translate the query into its formal mathematical expression ba

In [12]:
query = "Question: The overall probability of jyka is 40%. The probability of not jyka and kwox is 19%. The probability of jyka and kwox is 34%.Is the chance of kwox larger when observing jyka?\nGuidance: Address the question above by following the steps below:\nStep 1) Extract the causal graph: Identify the causal graph that depicts the relationships in the scenario. The diagram should simply consist of edges denoted in 'var1 -> var2' format, separated by commas. Step 2) Determine the query type: Identify the type of query implied by the main question. Choices include 'marginal probability', 'conditional probability', 'explaining away effect', 'backdoor adjustment set', 'average treatment effect', 'collider bias', 'normal counterfactual question', 'average treatment effect on treated', 'natural direct effect' or 'natural indirect effect'. Your answer should only be a term from the list above, enclosed in quotation marks. Step 3) Formalize the query: Translate the query into its formal mathematical expression based on its type, utilizing the 'do(·)' notation or counterfactual notations as needed."
text_gen = pipeline(task="text-generation", model=refined_model, tokenizer=llama_tokenizer, max_length=300)
output = text_gen(f"<s>[INST] {query} [/INST]")
print(output[0]['generated_text'])

/home/bazaluk/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

<s>[INST] Question: The overall probability of jyka is 40%. The probability of not jyka and kwox is 19%. The probability of jyka and kwox is 34%.Is the chance of kwox larger when observing jyka?
Guidance: Address the question above by following the steps below:
Step 1) Extract the causal graph: Identify the causal graph that depicts the relationships in the scenario. The diagram should simply consist of edges denoted in 'var1 -> var2' format, separated by commas. Step 2) Determine the query type: Identify the type of query implied by the main question. Choices include 'marginal probability', 'conditional probability', 'explaining away effect', 'backdoor adjustment set', 'average treatment effect', 'collider bias', 'normal counterfactual question', 'average treatment effect on treated', 'natural direct effect' or 'natural indirect effect'. Your answer should only be a term from the list above, enclosed in quotation marks. Step 3) Formalize the query: Translate the query into its formal 

In [16]:
text_gen(query)

[{'generated_text': "Question: The overall probability of jyka is 40%. The probability of not jyka and kwox is 19%. The probability of jyka and kwox is 34%.Is the chance of kwox larger when observing jyka?\nGuidance: Address the question above by following the steps below:\nStep 1) Extract the causal graph: Identify the causal graph that depicts the relationships in the scenario. The diagram should simply consist of edges denoted in 'var1 -> var2' format, separated by commas. Step 2) Determine the query type: Identify the type of query implied by the main question. Choices include 'marginal probability', 'conditional probability', 'explaining away effect', 'backdoor adjustment set', 'average treatment effect', 'collider bias', 'normal counterfactual question', 'average treatment effect on treated', 'natural direct effect' or 'natural indirect effect'. Your answer should only be a term from the list above, enclosed in quotation marks. Step 3) Formalize the query: Translate the query int